In [6]:
import requests
import json
from IPython.display import display
import pandas as pd

# San Francisco zone ID
sf_zone_id = "CAZ006"

# API endpoint for San Francisco forecast
url = f"https://api.weather.gov/zones/forecast/{sf_zone_id}/forecast"

# Make the request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    forecast_data = response.json()
    
    # Print basic information
    print(f"Forecast for: {forecast_data.get('properties', {}).get('name', 'Unknown')}")
    
    # Parse and display the periods of the forecast
    periods = forecast_data.get('properties', {}).get('periods', [])
    
    # Create a DataFrame for easier viewing
    forecast_df = pd.DataFrame([{
        'name': period.get('name'),
        'temperature': period.get('temperature'),
        'temperatureUnit': period.get('temperatureUnit'),
        'windSpeed': period.get('windSpeed'),
        'windDirection': period.get('windDirection'),
        'shortForecast': period.get('shortForecast'),
        'detailedForecast': period.get('detailedForecast')
    } for period in periods])
    
    display(forecast_df)
else:
    print(f"Error: {response.status_code}")
    print(response.text)


Forecast for: Unknown


,name,temperature,temperatureUnit,windSpeed,windDirection,shortForecast,detailedForecast
0,Rest Of Today,None,None,None,None,None,Partly cloudy. Highs in the mid 50s to lower 6...
1,Tonight,None,None,None,None,None,Mostly cloudy. Lows in the lower 50s. West win...
2,Thursday,None,None,None,None,None,Mostly cloudy. Highs in the mid to upper 50s. ...
3,Thursday Night,None,None,None,None,None,Cloudy. Lows in the lower 50s. West winds 10 t...
4,Friday,None,None,None,None,None,Mostly cloudy. Highs in the mid 50s to lower 6...
5,Friday Night,None,None,None,None,None,Mostly cloudy. A chance of rain. Lows around 50.
6,Saturday,None,None,None,None,None,Mostly cloudy. A chance of rain. Highs in the ...
7,Saturday Night,None,None,None,None,None,Partly cloudy in the evening then becoming mos...
8,Sunday,None,None,None,None,None,Mostly cloudy in the morning then becoming par...
9,Sunday Night through Tuesday,None,None,None,None,None,Mostly clear. Lows in the lower 50s. Highs in ...


In [9]:
import requests
import json
from datetime import datetime
import time

def get_sf_weather():
    """
    Function to get San Francisco weather forecast from NWS API
    """
    sf_zone_id = "CAZ006"
    url = f"https://api.weather.gov/zones/forecast/{sf_zone_id}/forecast"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data['properties']['periods']
        else:
            print(f"Error fetching weather: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

def check_weather_conditions(forecast_periods, conditions=None):
    """
    Check if specific weather conditions are present in the forecast
    
    Args:
        forecast_periods: List of forecast periods from the NWS API
        conditions: List of conditions to check for (e.g., 'rain', 'snow')
                   If None, will check for any notable conditions
    
    Returns:
        List of alerts about notable weather conditions
    """
    if conditions is None:
        conditions = ['rain', 'snow', 'storm', 'wind', 'fog']
    
    alerts = []
    
    for period in forecast_periods:
        period_name = period['name']
        forecast = period['detailedForecast'].lower()
        
        for condition in conditions:
            if condition.lower() in forecast:
                alerts.append({
                    'period': period_name,
                    'condition': condition,
                    'forecast': period['detailedForecast']
                })
                break  # Only report one condition per period
    
    return alerts

def send_slack_notification(webhook_url, alerts):
    """
    Send weather alerts to a Slack channel
    
    Args:
        webhook_url: Slack webhook URL
        alerts: List of weather alerts
    """
    if not alerts:
        return
    
    # Format the message
    blocks = [
        {
            "type": "header",
            "text": {
                "type": "plain_text",
                "text": "🌦️ San Francisco Weather Alert 🌦️"
            }
        },
        {
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"*Weather update as of {datetime.now().strftime('%Y-%m-%d %H:%M')}*"
            }
        }
    ]
    
    for alert in alerts:
        blocks.append({
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"*{alert['period']}*: {alert['forecast']}"
            }
        })
    
    message = {
        "blocks": blocks
    }
    
    try:
        response = requests.post(webhook_url, json=message)
        if response.status_code != 200:
            print(f"Error sending to Slack: {response.status_code}, {response.text}")
    except Exception as e:
        print(f"Exception sending to Slack: {e}")

# For testing in the notebook
forecast = get_sf_weather()
if forecast:
    # Display the current forecast
    for period in forecast[:2]:  # Just show first two periods
        print(f"{period['name']}: {period['detailedForecast']}")
    
    # Check for notable conditions
    alerts = check_weather_conditions(forecast)
    
    if alerts:
        print("\nNotable weather conditions:")
        for alert in alerts:
            print(f"{alert['period']}: {alert['condition']} - {alert['forecast']}")
    else:
        print("\nNo notable weather conditions detected.")

# To implement the scheduled job functionality, you'd add code like this:
"""
# Replace with your actual Slack webhook URL
SLACK_WEBHOOK_URL = "https://hooks.slack.com/services/YOUR/WEBHOOK/URL"

# For manual testing
# alerts = check_weather_conditions(get_sf_weather())
# if alerts:
#     send_slack_notification(SLACK_WEBHOOK_URL, alerts)

# To run every 4 hours (not ideal for a notebook, better in a script with proper scheduling)
while True:
    forecast = get_sf_weather()
    if forecast:
        alerts = check_weather_conditions(forecast)
        if alerts:
            send_slack_notification(SLACK_WEBHOOK_URL, alerts)
    
    # Sleep for 4 hours
    time.sleep(4 * 60 * 60)
"""


Rest Of Today: Partly cloudy. Highs in the mid 50s to lower 60s. West winds 10 to 15 mph.
Tonight: Mostly cloudy. Lows in the lower 50s. West winds 5 to 15 mph.

Notable weather conditions:
Rest Of Today: wind - Partly cloudy. Highs in the mid 50s to lower 60s. West winds 10 to 15 mph.
Tonight: wind - Mostly cloudy. Lows in the lower 50s. West winds 5 to 15 mph.
Thursday: wind - Mostly cloudy. Highs in the mid to upper 50s. West winds 10 to 20 mph.
Thursday Night: wind - Cloudy. Lows in the lower 50s. West winds 10 to 20 mph.
Friday: wind - Mostly cloudy. Highs in the mid 50s to lower 60s. West winds 5 to 15 mph.
Friday Night: rain - Mostly cloudy. A chance of rain. Lows around 50.
Saturday: rain - Mostly cloudy. A chance of rain. Highs in the mid 50s to lower 60s.


'\n# Replace with your actual Slack webhook URL\nSLACK_WEBHOOK_URL = "https://hooks.slack.com/services/YOUR/WEBHOOK/URL"\n\n# For manual testing\n# alerts = check_weather_conditions(get_sf_weather())\n# if alerts:\n#     send_slack_notification(SLACK_WEBHOOK_URL, alerts)\n\n# To run every 4 hours (not ideal for a notebook, better in a script with proper scheduling)\nwhile True:\n    forecast = get_sf_weather()\n    if forecast:\n        alerts = check_weather_conditions(forecast)\n        if alerts:\n            send_slack_notification(SLACK_WEBHOOK_URL, alerts)\n    \n    # Sleep for 4 hours\n    time.sleep(4 * 60 * 60)\n'

In [7]:
import requests

# Base URL for NWS API
base_url = "https://api.weather.gov"

# Search for zones containing "San Francisco"
search_url = f"{base_url}/zones"
params = {
    "type": "forecast",
    "area": "CA"  # California
}

response = requests.get(search_url, params=params)

if response.status_code == 200:
    zones = response.json()
    
    # Filter for San Francisco
    sf_zones = [
        zone for zone in zones['features'] 
        if 'San Francisco' in zone['properties']['name']
    ]
    
    for zone in sf_zones:
        print(f"ID: {zone['properties']['id']}")
        print(f"Name: {zone['properties']['name']}")
        print(f"State: {zone['properties'].get('state', 'N/A')}")
        print("-" * 30)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

ID: CAZ006
Name: San Francisco
State: CA
------------------------------
ID: CAZ508
Name: San Francisco Bay Shoreline
State: CA
------------------------------
ID: CAZ509
Name: San Francisco Peninsula Coast
State: CA
------------------------------


In [8]:
import requests
